# Playground

The notebook where random scripts and commands are placed.

In [ ]:
from ftplib import FTP
ftp = FTP('ftp.gportal.jaxa.jp')
ftp.login('', 'anonymous')

In [3]:
base = 'downloads'

import os
dir = "standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01"
os.makedirs(os.path.join(base, dir), exist_ok=True)

# Ascending data
filename_a = 'GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5'
with open(os.path.join(base, dir, filename_a), 'wb') as f:
    ftp.retrbinary(f"RETR /{dir}/{filename_a}", f.write)

# Descending data
filename_a = 'GC1SG1_20230901D01D_T0628_L2SG_LST_Q_3000.h5'
with open(os.path.join(base, dir, filename_a), 'wb') as f:
    ftp.retrbinary(f"RETR /{dir}/{filename_a}", f.write)

In [4]:
from osgeo import gdal
downloaded_dir = "downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01"
output_dir = "output/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01"
os.makedirs(output_dir, exist_ok=True)

# Ascending data
downloaded_filename_a = "GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"
downloaded_a = gdal.Open(os.path.join(downloaded_dir, downloaded_filename_a), gdal.GA_ReadOnly)

In [5]:
downloaded_a.GetMetadata()

{'Geometry_data_Grid_interval': '0.0020833334 ',
 'Geometry_data_Grid_interval_unit': 'deg',
 'Geometry_data_Image_projection': 'EQA (sinusoidal equal area) projection from 0-deg longitude',
 'Geometry_data_Lower_left_latitude': '20 ',
 'Geometry_data_Lower_left_longitude': '106.41778 ',
 'Geometry_data_Lower_right_latitude': '20 ',
 'Geometry_data_Lower_right_longitude': '117.05956 ',
 'Geometry_data_Number_of_lines': '4800 ',
 'Geometry_data_Number_of_pixels': '4800 ',
 'Geometry_data_Obs_time_Data_description': 'Observation time (hour)',
 'Geometry_data_Obs_time_Error_DN': '-128 ',
 'Geometry_data_Obs_time_Maximum_valid_DN': '127 ',
 'Geometry_data_Obs_time_Minimum_valid_DN': '-127 ',
 'Geometry_data_Obs_time_Offset': '14.593772 ',
 'Geometry_data_Obs_time_Resampling_interval': '1 ',
 'Geometry_data_Obs_time_Resampling_interval_unit': 'NA',
 'Geometry_data_Obs_time_Slope': '0.0065730005 ',
 'Geometry_data_Obs_time_Unit': 'hour',
 'Geometry_data_Upper_left_latitude': '30 ',
 'Geometr

In [6]:
downloaded_a.GetSubDatasets()

[('HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Geometry_data/Obs_time',
  '[4800x4800] //Geometry_data/Obs_time (8-bit signed character)'),
 ('HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Image_data/E01',
  '[4800x4800] //Image_data/E01 (8-bit character)'),
 ('HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Image_data/E02',
  '[4800x4800] //Image_data/E02 (8-bit character)'),
 ('HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Image_data/LST',
  '[4800x4800] //Image_data/LST (16-bit unsigned integer)'),
 ('HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Image_data/QA_flag',
  '[4800x4800] //Image_data/QA_flag (16-bit unsigned integer)')

In [7]:
downloaded_a.GetSubDatasets()[3][0]

'HDF5:"downloads/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.LST_/3/2023/09/01/GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.h5"://Image_data/LST'

In [8]:
image_ds_a = gdal.Open(downloaded_a.GetSubDatasets()[3][0], gdal.GA_ReadOnly)
image_ds_a.GetRasterBand(1).SetScale(0.02)
image_ds_a.GetRasterBand(1).SetOffset(0)

output_filename_a = "GC1SG1_20230901A01D_T0628_L2SG_LST_Q_3000.tif"
gdal.Translate(
    os.path.join(output_dir, output_filename_a),
    image_ds_a,
    format="GTiff",
    outputSRS="EPSG:4326",
    outputBounds=[-180, 90, 180, -90],
    noData=65535,
    creationOptions=['COMPRESS=LZW'])

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0xffff7fd6cff0> >